### Пример проекта по визуализации геоданных

Библиотеки, которые нам понадобились:

**pandas** для сбора и анализа данных
**geocoder** - для получения коорднат из названия стран
**folium** - для нанесения данных на карту

Установка:
в пустой ячейке введите:
!pip install geocoder folium


https://proglib.io/p/python-web-map/
пошаговая инструкция по фолиуму

In [1]:
!pip install geocoder folium

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r = requests.get('https://www.worldometers.info/coronavirus/')


In [4]:
df = pd.read_html('https://www.worldometers.info/coronavirus/')[0]

In [9]:
df[df['Country,Other'] == 'Kazakhstan']

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
96,Kazakhstan,67,+5,NaN,NaN,NaN,67,NaN,4.0


In [17]:
df[df['Country,Other'] == 'Russia']

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,lat,lng,ratio
44,Russia,438,NaN,1.0,NaN,17.0,420,NaN,3.0,52.12379,19.398769,0.23


In [32]:
# import geocoder
# lat = []
# lng = []
# for i in range(len(pickled)):
#     g = geocoder.arcgis(pickled['Country,Other'].iloc[i])
#     lat.append(g.json['lat'])
#     lng.append(g.json['lng'])

# pickled['lat'] = lat
# pickled['lng'] = lng

In [19]:
import geocoder
g = geocoder.arcgis('Russia')
g.json

{'address': 'Russia',
 'bbox': {'northeast': [84.9, 148.57137153400006],
  'southwest': [10.223526183000068, 45.039371534000075]},
 'confidence': 1,
 'lat': 61.989526183000066,
 'lng': 96.80537153400007,
 'ok': True,
 'quality': 'Locality',
 'raw': {'name': 'Russia',
  'extent': {'xmin': 45.039371534000075,
   'ymin': 10.223526183000068,
   'xmax': 148.57137153400006,
   'ymax': 84.9},
  'feature': {'geometry': {'x': 96.80537153400007, 'y': 61.989526183000066},
   'attributes': {'Score': 100, 'Addr_Type': 'Locality'}}},
 'score': 100,
 'status': 'OK'}

In [11]:
pickled = pd.read_csv("https://raw.githubusercontent.com/alnagaev/python_lessons/master/latlng.csv")
df['lat'] = pickled['lat']
df['lng'] = pickled['lng'] 

In [12]:
df['ratio'] = round(df['TotalDeaths']/df['TotalCases']*100, 2)
df[['Country,Other', 'ratio', ]].dropna().head()

,"Country,Other",ratio
0,China,4.04
1,Italy,9.51
2,USA,1.26
3,Spain,6.58
4,Germany,0.42


In [13]:
from math import ceil
def get_radius(rat):
    if rat > 10:
        return 10
    elif rat < 1:
        return 1
    else:
        return ceil(rat)

In [14]:
dropped = df[['Country,Other', 'ratio', 'lat', 'lng', 'TotalCases']].dropna()
dropped['percent'] = round(dropped['TotalCases']/dropped['TotalCases'].sum(), 2)*100
dropped['percent'].head()

0    22.0
1    17.0
2    12.0
3     9.0
4     8.0
Name: percent, dtype: float64

In [15]:
import folium
fol_map = folium.Map(location=[36.567348, 103.930027], zoom_start=2)
for lat, lon, percent, total, country in zip(
                            dropped['lat'].tolist(), 
                            dropped['lng'].tolist(), 
                            dropped['percent'].tolist(),
                            dropped['TotalCases'].tolist(),
                            dropped['Country,Other'].tolist()
                                 ):
    folium.CircleMarker(location=[lat, lon], radius = get_radius(percent), popup= str(country)+' ' + str(total)+" total cases").add_to(fol_map)

In [16]:
fol_map